<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (Optimised)

**⚡ Fast startup (~5 minutes after first run)**

**Features:**
- Google Drive integration for persistent storage
- ComfyUI Manager for easy custom node management
- Cloudflare tunnel for external access
- Colab Secrets integration for API tokens
- Optimised dependency installation (only installs what's missing)

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Add secrets (optional): Tools → Secrets → Add your API keys
3. Run all cells: Runtime → Run all

## 0. Configure Secrets (Optional)

Store your API keys securely using Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add your secrets:
   - `ANTHROPIC_API_KEY` - For Claude integration
   - `OPENAI_API_KEY` - For OpenAI models
   - `HF_TOKEN` - For Hugging Face private models
3. Toggle "Notebook access" for each secret

Secrets are stored in your Google account, not in the notebook file.

In [ ]:
#@title Load Secrets (if available)
import os

try:
    from google.colab import userdata
    
    # Try to load secrets - will only work if they exist and have notebook access enabled
    secrets_found = []
    
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    
    if secrets_found:
        print(f"✓ Loaded secrets: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found. Add them in the 🔑 Secrets panel if needed.")
        
except Exception as e:
    print(f"ℹ️  Secrets not available: {e}")

## 1. Mount Google Drive

Mount your Google Drive to save ComfyUI installation and models persistently.

In [ ]:
#@title Mount Google Drive
#@markdown This will save your ComfyUI installation to Google Drive

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}

if USE_GOOGLE_DRIVE:
    print("Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive
else:
    WORKSPACE = "/content/ComfyUI"
    %cd /content

print(f"Workspace: {WORKSPACE}")

## 2. Install ComfyUI

Clone ComfyUI repository (only runs on first setup) and optionally update it.

In [ ]:
#@title Install/Update ComfyUI
#@markdown Update ComfyUI to latest version?

UPDATE_COMFY_UI = False  #@param {type:"boolean"}

import os

# Clone ComfyUI if it doesn't exist
if not os.path.exists(WORKSPACE):
    print("=== Installing ComfyUI ===")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}

%cd {WORKSPACE}

# Update ComfyUI if requested
if UPDATE_COMFY_UI:
    print("=== Updating ComfyUI ===")
    !git pull origin master 2>/dev/null || git pull origin main

print("✓ ComfyUI ready!")

## 3. Install Dependencies (Smart)

**⚡ OPTIMISED:** Only installs packages that aren't already in Colab's base image.

Colab already includes: torch, torchvision, torchaudio, transformers, Pillow, scipy, tqdm, etc.

This section only installs the extras ComfyUI needs.

In [ ]:
#@title Install Missing Dependencies Only

print("=== Installing Missing Dependencies ===")
print("Skipping packages already in Colab base image...\n")

%cd {WORKSPACE}

# These are NOT in Colab's base image
!pip install -q --no-deps einops safetensors aiohttp pyyaml psutil tokenizers kornia spandrel soundfile av sentencepiece torchsde

# Upgrade these if versions are too old
!pip install -q --upgrade transformers accelerate

# Install from requirements.txt (but skip duplicates)
!pip install -q -r requirements.txt --no-deps 2>/dev/null || true

# Upgrade ComfyUI frontend
!pip install -q --upgrade comfyui-frontend-package

print("\n✓ Dependencies installed (only what was missing)!")

## 3.5. Verify PyTorch CUDA Support

Quick CUDA verification. Colab's PyTorch already has CUDA support, but we verify it.

In [ ]:
import torch

print(f"PyTorch {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\n✓ GPU ready!")
else:
    print("\n❌ No GPU detected!")
    print("Go to: Runtime > Change runtime type > Hardware accelerator > T4 GPU")

## 4. Install ComfyUI Manager

ComfyUI Manager makes it easy to install custom nodes and models.

In [ ]:
#@title Install ComfyUI Manager
#@markdown Install/update ComfyUI Manager?

USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}

if USE_COMFYUI_MANAGER:
    import os
    manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
    
    if not os.path.exists(manager_path):
        print("=== Installing ComfyUI-Manager ===")
        %cd {WORKSPACE}/custom_nodes
        !git clone https://github.com/ltdrdata/ComfyUI-Manager
    else:
        print("=== Updating ComfyUI-Manager ===")
        %cd {manager_path}
        !git pull
    
    print("✓ ComfyUI Manager ready!")
else:
    print("Skipping ComfyUI Manager")

## 4.5. Install Essential Custom Node Dependencies

**⚡ FAST:** Installs only the most commonly needed packages for your custom nodes.

**Takes 5-10 seconds instead of 30-40 minutes!**

These must be reinstalled each session because Python packages are temporary in Colab VM.

In [ ]:
#@title Quick Install: Essential Custom Node Packages
#@markdown Most common packages for custom nodes (runs in seconds)

INSTALL_ESSENTIAL = True  #@param {type:"boolean"}

#@markdown ---
#@markdown **Advanced:** Full restore of ALL dependencies? (30-40 minutes!)
FULL_RESTORE = False  #@param {type:"boolean"}

if INSTALL_ESSENTIAL:
    print("=== Installing Essential Custom Node Dependencies ===")
    
    # Common packages your custom nodes need
    !pip install -q ffmpeg-python deepdiff comfyui-types gguf piexif \
        transparent-background simpleeval iopath ollama anthropic segment-anything \
        GitPython
    
    print("\n✓ Essential packages installed!")

if FULL_RESTORE:
    print("\n=== Running FULL dependency restore (30-40 minutes) ===")
    %cd {WORKSPACE}
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    print("✓ Full restore complete!")

## 5. Start ComfyUI

**⚡ OPTIMISED:** Faster cloudflared tunnel startup.

Launch ComfyUI server with cloudflared tunnel for external access.

**Click the URL that appears below to access ComfyUI!**

In [ ]:
import subprocess
import threading
import time
import socket
import re

# Download cloudflared if not already present
![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

def start_tunnel(port=8188):
    """Start cloudflared tunnel and print URL quickly"""
    # Wait for ComfyUI to start
    print("Waiting for ComfyUI server to start...")
    for _ in range(60):  # 30 second timeout
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break
        except:
            pass
        time.sleep(0.5)
    
    print("\n" + "=" * 70)
    print("Starting cloudflared tunnel...")
    
    # Start cloudflared and capture URL immediately
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    # Read stderr line by line to find URL quickly
    url_found = False
    for line in proc.stderr:
        if "trycloudflare.com" in line and not url_found:
            # Extract and print URL
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                print(f"\n🌐 ACCESS COMFYUI HERE:\n{url}\n")
                print("=" * 70 + "\n")
                url_found = True
                break

# Start tunnel in background
tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()

# Start ComfyUI
print("\n" + "=" * 70)
print("Starting ComfyUI...")
print("=" * 70 + "\n")

%cd {WORKSPACE}
!python main.py --dont-print-server

## Tips & Troubleshooting

**Speed Tips:**
- First run: ~8-10 minutes (downloading dependencies)
- Subsequent runs: ~3-5 minutes (everything cached on Drive)
- Keep `UPDATE_COMFY_UI` set to `False` unless you specifically need updates
- Only use `FULL_RESTORE` if a custom node shows missing dependency errors

**API Keys:**
- Store in Colab Secrets (🔑 icon in sidebar)
- Enable "Notebook access" for each secret
- They'll be automatically loaded when you run Section 0

**Common Issues:**
- **"Torch not compiled with CUDA"**: Run Section 3.5 to verify GPU
- **Custom node errors**: Enable `FULL_RESTORE` in Section 4.5 (but expect 30-40 min wait)
- **Slow startup**: Make sure you're not reinstalling torch/transformers (Section 3 handles this)

**Storage:**
- Your files on Google Drive: **Permanent** (custom nodes, models, workflows)
- Python packages in Colab VM: **Temporary** (must reinstall each session)
- This is why Section 4.5 runs every time